In [1]:
from src.spark import Spark
import src.test as funcs
import matplotlib.pyplot as plt
from pyspark.sql import functions as fs


In [2]:
spark = Spark('load', 'spark://pc5-035-l.cs.st-andrews.ac.uk:7077')
sess = spark.session()

In [3]:
df = funcs.load_dataframe(sess, funcs.default_loc, funcs.schema)
df.first()

Row(text=u'RT @VinnyLingham2x: #Bitcoin needs less developers, and more incumbents and intermediaries! https://t.co/iia0k7i09B', entities=Row(hashtags=[Row(indices=[20, 28], text=u'Bitcoin')]), created_at=u'Thu Oct 26 06:26:17 +0000 2017', favourite_count=None, retweet_count=0, id_str=u'923435597242478592', user=Row(followers_count=59, utc_offset=None))

In [5]:
df2 = funcs.parse_timestmap(df)

df3 = df2.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count()

volume_df = df3.toPandas()

In [6]:
volume_df

,date,count
0,2017-10-20,256370
1,2017-10-31,152224
2,2017-11-01,290947
3,2017-10-24,125656
4,2017-10-22,307229
5,None,2
6,2017-11-10,214538
7,2017-11-23,215200
8,2017-11-16,442745
9,2017-11-14,382167


In [8]:
import os 
import pandas as pd
crix = pd.read_csv(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto", "crix.csv"), index_col=0)
crix.index = pd.to_datetime(crix.index)


In [ ]:
volume_df.dropna()
volume_df.set_index('date', inplace=True)
volume_df.index = pd.to_datetime(volume_df.index)

crix_and_vol = crix
crix_and_vol['volume'] = volume_df['count']
crix_and_vol_nona = crix_and_vol.dropna()

In [15]:
crix_and_vol_nona.corr()

,price,volume
price,1.000000,0.585943
volume,0.585943,1.000000


In [37]:
import matplotlib as mpl
import matplotlib.patches as mpatches
mpl.rc('font', size=20)          # controls default text sizes
mpl.rc('axes', labelsize=20)    # fontsize of the x and y labels
mpl.rc('xtick', labelsize=20)    # fontsize of the tick labels
mpl.rc('ytick', labelsize=20)    # fontsize of the tick labels
mpl.rc('legend', fontsize=14)    # legend fontsize
mpl.rc('figure', titlesize=20)  #
mpl.rc('lines', linewidth=1.5)
mpl.rc('figure', figsize=(16, 9))


def double_plot(plots, names, xy_labels, title, xs=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    line_style = "-"
    linewidth = 1.5
    ax.plot(xs, plots[0], linestyle=line_style, linewidth=linewidth, color="red") if xs is not None else ax.plot(plots[0])
    ax.set_ylabel(xy_labels[1])
    
    ax2 = fig.add_subplot(111, sharex=ax, frameon=False)
    ax2.plot(xs, plots[1], linestyle=line_style, linewidth=linewidth) if xs is not None else ax.plot(plot[0])
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position("right")
    ax2.set_ylabel(xy_labels[2])
    
    plt.title(title)
    plt.xlabel(xy_labels[0])

    red_patch = mpatches.Patch(color='red', label=names[0])
    blue = mpatches.Patch(color='blue', label=names[1])
    plt.legend(handles=[red_patch, blue], loc='upper left')
    plt.tight_layout()
    ax.get_yaxis().set_major_formatter(
        mpl.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))

    for (i, label) in enumerate(ax.get_xticklabels()):
        if 0 < i < len(ax.get_xticklabels()) - 1:
            label.set_visible(False)

    plt.show()
    plt.close()




In [39]:
scaled_price = crix_and_vol_nona.price * (crix_and_vol_nona.volume.iloc[0] / crix_and_vol_nona.price.iloc[0])
double_plot([crix_and_vol_nona.price, crix_and_vol_nona.volume], ['Index Price', 'Tweets'], ['Date', 'Price', 'Volume'], "Price vs Tweet Volume", crix_and_vol_nona.index.tolist())

In [42]:
crix_and_vol_nona.corr().price.volume


0.58594334311745255

In [47]:
bitcoin_hashes = ['btc', 'bitcoin']
df2.take(5)

[Row(text=u'RT @VinnyLingham2x: #Bitcoin needs less developers, and more incumbents and intermediaries! https://t.co/iia0k7i09B', entities=Row(hashtags=[Row(indices=[20, 28], text=u'Bitcoin')]), created_at=u'Thu Oct 26 06:26:17 +0000 2017', favourite_count=None, retweet_count=0, id_str=u'923435597242478592', user=Row(followers_count=59, utc_offset=None), timestamp=datetime.datetime(2017, 10, 26, 7, 26, 17)),
 Row(text=u'RT @LisaCiarlone: What the FORK!!! Bitcoin Cash Community Preps Hard Fork Slated for November 13! https://t.co/qCzqavmucQ', entities=Row(hashtags=[]), created_at=u'Thu Oct 26 06:26:17 +0000 2017', favourite_count=None, retweet_count=0, id_str=u'923435597267550208', user=Row(followers_count=0, utc_offset=None), timestamp=datetime.datetime(2017, 10, 26, 7, 26, 17)),
 Row(text=u'RT @LisaCiarlone: Are there any Politicians in the US that show support for Crypto-currency?  Please comment if you know of any. Than\u2026 ', entities=Row(hashtags=[]), created_at=u'Thu Oct 26 06:

In [118]:
btc_df = df2.withColumn("lowertext", fs.lower(fs.col("text")))
btc_df.show(1)

+--------------------+--------------------+--------------------+---------------+-------------+------------------+-----+-------------------+--------------------+
|                text|            entities|          created_at|favourite_count|retweet_count|            id_str| user|          timestamp|           lowertext|
+--------------------+--------------------+--------------------+---------------+-------------+------------------+-----+-------------------+--------------------+
|RT @VinnyLingham2...|[[[[20, 28], Bitc...|Thu Oct 26 06:26:...|           null|            0|923435597242478592|[59,]|2017-10-26 07:26:17|rt @vinnylingham2...|
+--------------------+--------------------+--------------------+---------------+-------------+------------------+-----+-------------------+--------------------+
only showing top 1 row



In [103]:
btc_df_ = btc_df.filter(fs.lower(df['text']).like("%bitcoin%") | fs.lower(df['text']).like("%btc%"))

In [104]:
btc_df_vol = btc_df_.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count().toPandas()

In [105]:
btc_df_vol

,date,count
0,2017-10-20,129752
1,2017-10-31,75257
2,2017-11-01,147054
3,2017-10-24,63104
4,2017-10-22,130149
5,2017-11-10,98193
6,2017-11-23,81247
7,2017-11-16,182179
8,2017-11-14,163515
9,2017-11-15,171196


In [113]:
btc = pd.read_csv(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto/KRAKEN_BTC_EUR.csv"), index_col=0)
btc.index = pd.to_datetime(btc.index)

btc_df_vol.dropna()
btc_df_vol.index = pd.to_datetime(btc_df_vol.index)

btc_and_vol = btc
btc_and_vol['price'] = btc_and_vol.weightedAverage
btc_and_vol['volume'] = btc_df_vol['count']
btc_and_vol_nona = btc_and_vol.dropna()

double_plot([btc_and_vol_nona.price, btc_and_vol_nona.volume], ['BTC Price', 'Tweets'], ['Date', 'Price', 'Volume'], "BTC Price vs Tweet Volume", btc_and_vol_nona.index.tolist())

In [111]:
btc_and_vol_nona.corr().price.volume

0.30750634486637513

In [119]:
eth_df = btc_df.filter(fs.lower(df['text']).like("%ether%") | fs.lower(df['text']).like("%eth%") | fs.lower(df['text']).like("%ethereum%"))
eth_df_vol = eth_df.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count().toPandas()

In [124]:
eth = pd.read_csv(os.path.abspath(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto","KRAKEN_ETH_EUR.csv")), index_col=0)
eth.index = pd.to_datetime(eth.index)

eth_df_vol.dropna()
eth_df_vol.set_index('date', inplace=True)
eth_df_vol.index = pd.to_datetime(eth_df_vol.index)

eth_and_vol = eth
eth_and_vol['price'] = eth_and_vol.weightedAverage
eth_and_vol['volume'] = eth_df_vol['count']
eth_and_vol_nona = eth_and_vol.dropna()

double_plot([eth_and_vol_nona.price, eth_and_vol_nona.volume], ['ETH Price', 'Tweets'], ['Date', 'Price', 'Volume'], "ETH Price vs Tweet Volume", eth_and_vol_nona.index.tolist())

In [123]:
eth_df_vol

,date,count
1970-01-01 00:00:00.000000000,2017-10-20,33089
1970-01-01 00:00:00.000000001,2017-10-31,17956
1970-01-01 00:00:00.000000002,2017-11-01,37185
1970-01-01 00:00:00.000000003,2017-10-24,17504
1970-01-01 00:00:00.000000004,2017-10-22,38421
1970-01-01 00:00:00.000000005,2017-11-10,24914
1970-01-01 00:00:00.000000006,2017-11-23,27057
1970-01-01 00:00:00.000000007,2017-11-16,49582
1970-01-01 00:00:00.000000008,2017-11-14,45862
1970-01-01 00:00:00.000000009,2017-11-15,48066


In [125]:
eth_and_vol_nona.corr().price.volume

0.39946022045860696